In [48]:
from datetime import datetime
import time

def date2seconds(str_time):
    datetime_obj = datetime.strptime(str_time, '%Y-%m-%d %H:%M:%S')
    seconds = time.mktime(datetime_obj.timetuple())
    return seconds


def seconds2date(seconds):
    date_obj = time.localtime(seconds)
    str_time = '%04d-%02d-%02d %02d:%02d:%02d'
    return str_time % (date_obj.tm_year, date_obj.tm_mon, date_obj.tm_mday,
                       date_obj.tm_hour, date_obj.tm_min, date_obj.tm_sec) 

In [49]:
# from rltensor.environments.utils import date2seconds

end = '2016-09-07 04:00:00'
sec = date2seconds(end)
print(sec)
d = seconds2date(sec)
print(d)

1473235200.0
2016-09-07 04:00:00


In [35]:
sec = date2seconds(d)
print(sec)
d = seconds2date(sec)
print(d)

1473580800.0
2016-09-11 08:00:00


In [6]:
t1

time.struct_time(tm_year=2010, tm_mon=9, tm_mday=12, tm_hour=10, tm_min=19, tm_sec=54, tm_wday=6, tm_yday=255, tm_isdst=0)

In [9]:
'%02d' % 12

'12'